In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn import linear_model
from sklearn.linear_model import LinearRegression





#可以用決策數分析來協助報告

In [33]:
#資料讀入以及合併
ls3 = pd.read_csv("train.csv",encoding="BIG5")
ls3
repet = ls3[ls3["Y1"]=="Y"]
ls3 = pd.concat([ls3,repet,repet,repet,repet,repet])# 
#記得調整
num_rep = 2000*5
#記得調整
all_classes = ls3['Y1'].values
i = 0
while i < len(all_classes):
    if all_classes[i] == "Y":
        all_classes[i] = 1
    else:
        all_classes[i] = 0
    i +=1
ls3=ls3.drop("Y1",axis=1)

ls2 = pd.read_csv("test.csv",encoding="BIG5")
name = ls2["CUS_ID"]
ls = pd.concat([ls3,ls2])  #最後調整好的是ls 

c:\program files\python37\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (17,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [34]:
need_to_0 = ["DIEBENEFIT_AMT","DIEACCIDENT_AMT","POLICY_VALUE_AMT","ANNUITY_AMT","EXPIRATION_AMT","ACCIDENT_HOSPITAL_REC_AMT",
        "DISEASES_HOSPITAL_REC_AMT","OUTPATIENT_SURGERY_AMT","INPATIENT_SURGERY_AMT","PAY_LIMIT_MED_MISC_AMT","FIRST_CANCER_AMT",
        "ILL_ACCELERATION_AMT","ILL_ADDITIONAL_AMT","LONG_TERM_CARE_AMT","MONTHLY_CARE_AMT",
        ]

need_to_N = ["IF_ISSUE_INSD_A_IND","IF_ISSUE_INSD_B_IND","IF_ISSUE_INSD_C_IND","IF_ISSUE_INSD_D_IND","IF_ISSUE_INSD_E_IND",
        "IF_ISSUE_INSD_F_IND","IF_ISSUE_INSD_G_IND","IF_ISSUE_INSD_H_IND","IF_ISSUE_INSD_I_IND","IF_ISSUE_INSD_J_IND",
        "IF_ISSUE_INSD_K_IND","IF_ISSUE_INSD_L_IND","IF_ISSUE_INSD_M_IND","IF_ISSUE_INSD_N_IND","IF_ISSUE_INSD_O_IND",
        "IF_ISSUE_INSD_P_IND","IF_ISSUE_INSD_Q_IND",
        "IF_ADD_INSD_F_IND","IF_ADD_INSD_L_IND","IF_ADD_INSD_Q_IND","IF_ADD_INSD_G_IND","IF_ADD_INSD_R_IND",    
        "IF_ADD_INSD_IND",
        "FINANCETOOLS_A","FINANCETOOLS_B","FINANCETOOLS_C","FINANCETOOLS_D","FINANCETOOLS_E","FINANCETOOLS_F","FINANCETOOLS_G",
        "X_A_IND","X_B_IND","X_C_IND","X_D_IND","X_E_IND","X_F_IND","X_G_IND","X_H_IND",
            'A_IND', 'B_IND', 'C_IND']
for i in need_to_0 :
    ls[i][ls[i].isna()] = 0
for i in need_to_N :
    ls[i][ls[i].isna()] = "N"
ls["GENDER"][ls["GENDER"].isna()] = "F"#性別 填女
ls["OCCUPATION_CLASS_CD"][ls["OCCUPATION_CLASS_CD"].isna()] = 1# 職業風險分數 眾數
ls["TERMINATION_RATE"][ls["TERMINATION_RATE"].isna()] = 0# 曾經解保單比率 眾數為0 加上值觀上也算是沒有解保單過
ls["LEVEL"][ls["LEVEL"].isna()] = 0  #往來等級 1~5 但沒有往來 所以決定填0
ls["RFM_M_LEVEL"][ls["RFM_M_LEVEL"].isna()] = 1#曾投保主約級數 3~10 無投保 推估1
ls["APC_1ST_AGE"][ls["APC_1ST_AGE"].isna()] = "無"        #第一次擔任被保人年齡(等級)
ls["RFM_R"][ls["RFM_R"].isna()] = "無"                    #上次要保人身分投保  (等級)這三個獨立出來當一個類別
ls["REBUY_TIMES_CNT"][ls["REBUY_TIMES_CNT"].isna()] = "無"#再次購買           (等級)

#這段是對教育程度和年收入的交叉填值
new = ls[["EDUCATION_CD","ANNUAL_INCOME_AMT"]]
b = new.groupby("EDUCATION_CD").mean().unstack().unstack()
new["ANNUAL_INCOME_AMT"][(new["ANNUAL_INCOME_AMT"].isna()) & (new["EDUCATION_CD"]==1)]=float(b[1])
new["ANNUAL_INCOME_AMT"][(new["ANNUAL_INCOME_AMT"].isna()) & (new["EDUCATION_CD"]==2)]=float(b[2])
new["ANNUAL_INCOME_AMT"][(new["ANNUAL_INCOME_AMT"].isna()) & (new["EDUCATION_CD"]==3)]=float(b[3])
new["ANNUAL_INCOME_AMT"][(new["ANNUAL_INCOME_AMT"].isna()) & (new["EDUCATION_CD"]==4)]=float(b[4])
new["ANNUAL_INCOME_AMT"][(new["ANNUAL_INCOME_AMT"].isna())] = 0.000613 #甚麼都沒有的就填平均
new["EDUCATION_CD"][(new["EDUCATION_CD"].isna()) & (new["ANNUAL_INCOME_AMT"]<=0.000354)] = 1.0
new["EDUCATION_CD"][(new["EDUCATION_CD"].isna()) & (new["ANNUAL_INCOME_AMT"]<=0.000541)] = 2.0
new["EDUCATION_CD"][(new["EDUCATION_CD"].isna()) & (new["ANNUAL_INCOME_AMT"]<=0.000627)] = 3.0
new["EDUCATION_CD"][(new["EDUCATION_CD"].isna()) & (new["ANNUAL_INCOME_AMT"]<=1)] = 4.0
ls["EDUCATION_CD"] = new["EDUCATION_CD"]
ls["ANNUAL_INCOME_AMT"] = new["ANNUAL_INCOME_AMT"]
#填年度化繳費
ls["ANNUAL_PREMIUM_AMT"][ls["ANNUAL_PREMIUM_AMT"].isna()]=ls["L1YR_GROSS_PRE_AMT"][ls["ANNUAL_PREMIUM_AMT"].isna()] 
#填結婚的na
ls["MARRIAGE_CD"][(ls["MARRIAGE_CD"].isna()) & (ls["INSD_CNT"] == 0)] = 0
ls["MARRIAGE_CD"][ls["MARRIAGE_CD"].isna()] = 1


c:\program files\python37\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\program files\python37\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\program files\python37\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\program files\python37\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [35]:
#資料處理
check=ls.isna()
column =ls.columns.values.tolist()
column.remove("BMI")
k = 0

for i in column:
    if sum(check[i]) >0:  #將有缺失值
        ls = ls.drop(i,axis=1)
        print(i)
        k += 1
ls=ls.replace("無",0)
ls=ls.replace("低",1)#將可量化的值取代
ls=ls.replace("中",2)
ls=ls.replace("中高",3)
ls=ls.replace("高",4)

ls = pd.get_dummies(ls)# 將剩餘不可量化、單純為類別的變數轉換為1000、0100的格式
#ls=ls.drop("Y1_N",axis=1)#上一步驟的轉換會多轉一個沒有意義的column 將這個刪掉
ls=ls.drop("CUS_ID",axis=1)#流水號刪除
b = ls.columns.values.tolist()#b=所有column的名子的串列
print(k)

#以下為使用全部數值預測BMI
print(sum(ls["BMI"] ==0))
b.remove("BMI")
train_inputs = ls[b][ls["BMI"]>=0]
train_classes = ls["BMI"][ls["BMI"]>=0]
test_inputs = ls[b][ls["BMI"].isna()]
train_classes = np.array(train_classes)
dtc = LinearRegression()
dtc.fit(train_inputs, train_classes)
a = dtc.predict(test_inputs)
ls["BMI"][ls["BMI"].isna()] = a
b = ls.columns.values.tolist()
print(sum(ls["BMI"] ==0))

INSD_1ST_AGE
APC_1ST_YEARDIF
L1YR_C_CNT
INSD_LAST_YEARDIF_CNT
4
160
160


c:\program files\python37\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# 以上為資料預處理

# 以下為資料分析

In [36]:
#分割資料為測試集#這是拿來測試的
all_inputs = ls[b].values[:100000+num_rep]
all_classes = np.array(all_classes, dtype=int)
(train_inputs, test_inputs, train_classes, test_classes) = train_test_split(all_inputs, all_classes, train_size=0.7, random_state=1)

In [19]:
from sklearn.kernel_approximation import RBFSampler#調整參數用
from sklearn.naive_bayes import GaussianNB
from sklearn.calibration import CalibratedClassifierCV


In [ ]:
"""
linear_svc = LinearSVC()

clf = linear_svc.fit(X_train_transformed,y_train_lables_trf)

calibrated_svc = CalibratedClassifierCV(base_estimator=linear_svc,

cv="prefit")

calibrated_svc.fit(X_train_transformed,y_train_lables_trf)

predicted = calibrated_svc.predict(X_test_transformed)
calibrated_svc.predict_proba(p_tfidf)
"""

In [37]:
#voting classifier 
dtc = LinearSVC(C=1.0, class_weight='balanced', dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
#dtc = linear_model.SGDClassifier()
#dtc = GaussianNB()
#dtc = BaggingClassifier()
dtc.fit(train_inputs, train_classes)
#算機率
calibrated_svc = CalibratedClassifierCV(base_estimator=dtc,method = "sigmoid" ,cv="prefit")
#clf.calibrated_svc = CalibratedClassifierCV(base_estimator=dtc)
calibrated_svc.fit(train_inputs, train_classes)
print(dtc.score(test_inputs, test_classes))
print(calibrated_svc.score(test_inputs, test_classes))




c:\program files\python37\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.8411818181818181
0.891090909090909


In [38]:
calibrated_svc.predict_proba(test_inputs)

array([[0.91632779, 0.08367221],
       [0.85762017, 0.14237983],
       [0.84415936, 0.15584064],
       ...,
       [0.94127452, 0.05872548],
       [0.98648432, 0.01351568],
       [0.94640813, 0.05359187]])

In [39]:
sum(sum(dtc.coef_>0.15))

19

In [40]:
dtc.intercept_

array([-0.00785838])

In [41]:
c = calibrated_svc.predict(test_inputs)
sum(c)

507

In [42]:
cc = dtc.predict(test_inputs)
sum(cc)

4600

In [43]:
c = cc

## 測試結果

In [44]:
print("總預測數 %d "%len(c))
print("預測有續約%d "%sum(test_classes == 1))
print("預測無續約 %d "%sum(test_classes == 0))
print("有續約且預測為有續約數:%d"%sum((c == test_classes)&(test_classes == 1)))
print("無續約且預測為無續約數:%d"%sum((c == test_classes)&(test_classes == 0)))
print("有續約但預測為無續約數:%d"%sum((c != test_classes)&(test_classes == 1)))
print("無續約但預測為有續約數:%d"%sum((c != test_classes)&(test_classes == 0)))
print("有續約且預測為有續約率:%f"%(sum((c == test_classes)&(test_classes == 1))/sum(test_classes == 1)))
print("無續約且預測為無續約率:%f"%(sum((c == test_classes)&(test_classes == 0))/sum(test_classes == 0)))
print("有續約但預測為無續約率:%f"%(sum((c != test_classes)&(test_classes == 1))/sum(test_classes == 1)))
print("無續約但預測為有續約率:%f"%(sum((c != test_classes)&(test_classes == 0))/sum(test_classes == 0)))
print("正確率%f"%(sum(c == test_classes)/len(test_classes)))

總預測數 33000 
預測有續約3617 
預測無續約 29383 
有續約且預測為有續約數:1488
無續約且預測為無續約數:26271
有續約但預測為無續約數:2129
無續約但預測為有續約數:3112
有續約且預測為有續約率:0.411391
無續約且預測為無續約率:0.894088
有續約但預測為無續約率:0.588609
無續約但預測為有續約率:0.105912
正確率0.841182


In [ ]:
c = dtc.predict(test_inputs)
sum(c)
#sum(test_classes)

In [ ]:
print("總預測數 %d "%len(c))
print("預測有續約%d "%sum(test_classes == 1))
print("預測無續約 %d "%sum(test_classes == 0))
print("有續約且預測為有續約數:%d"%sum((c == test_classes)&(test_classes == 1)))
print("無續約且預測為無續約數:%d"%sum((c == test_classes)&(test_classes == 0)))
print("有續約但預測為無續約數:%d"%sum((c != test_classes)&(test_classes == 1)))
print("無續約但預測為有續約數:%d"%sum((c != test_classes)&(test_classes == 0)))
print("有續約且預測為有續約率:%f"%(sum((c == test_classes)&(test_classes == 1))/sum(test_classes == 1)))
print("無續約且預測為無續約率:%f"%(sum((c == test_classes)&(test_classes == 0))/sum(test_classes == 0)))
print("有續約但預測為無續約率:%f"%(sum((c != test_classes)&(test_classes == 1))/sum(test_classes == 1)))
print("無續約但預測為有續約率:%f"%(sum((c != test_classes)&(test_classes == 0))/sum(test_classes == 0)))
print("正確率%f"%(sum(c == test_classes)/len(test_classes)))

In [ ]:
#輸入值 : ls

#模型  :  dtc = LinearSVC(intercept_scaling=1,class_weight="balanced")

#結果
"""
總預測數 30000 
預測有續約603 
預測無續約 29397 
有續約且預測為有續約數:127
無續約且預測為無續約數:28420
有續約但預測為無續約數:476
無續約但預測為有續約數:977
有續約且預測為有續約率:0.210614
無續約且預測為無續約率:0.966765
有續約但預測為無續約率:0.789386
無續約但預測為有續約率:0.033235
正確率0.951567
"""

In [ ]:
#輸入值 :  ls+複製一次有續約

#模型 :  dtc = LinearSVC(intercept_scaling=1,class_weight="balanced")
#結果
"""
總預測數 30600 
預測有續約1195 
預測無續約 29405 
有續約且預測為有續約數:384
無續約且預測為無續約數:27058
有續約但預測為無續約數:811
無續約但預測為有續約數:2347
有續約且預測為有續約率:0.321339
無續約且預測為無續約率:0.920184
有續約但預測為無續約率:0.678661
無續約但預測為有續約率:0.079816
正確率0.896797
"""

In [ ]:
#輸入值 :  ls+複製兩次有續約

#模型 :  dtc = LinearSVC(intercept_scaling=1,class_weight="balanced")
#結果
"""
總預測數 31200 
預測有續約1797 
預測無續約 29403 
有續約且預測為有續約數:859
無續約且預測為無續約數:25452
有續約但預測為無續約數:938
無續約但預測為有續約數:3951
有續約且預測為有續約率:0.478019
無續約且預測為無續約率:0.865626
有續約但預測為無續約率:0.521981
無續約但預測為有續約率:0.134374
正確率0.843301
"""

In [ ]:
#模型 :  dtc = LinearSVC(intercept_scaling=1,class_weight="balanced")
#結果(5個)
""""""
#總預測數 33000 
#預測有續約3617 
#預測無續約 29383 
#有續約且預測為有續約數:1488
#無續約且預測為無續約數:26271
#有續約但預測為無續約數:2129
#無續約但預測為有續約數:3112
#有續約且預測為有續約率:0.411391
#無續約且預測為無續約率:0.894088
#有續約但預測為無續約率:0.588609
#無續約但預測為有續約率:0.105912
#正確率0.841182
""""""

## 加入test

In [ ]:
all_inputs = ls[b][:100000+num_rep].values
all_classes = np.array(all_classes, dtype=int)
#all_classes = ls['Y1_Y'].values
#(train_inputs, test_inputs, train_classes, test_classes) = train_test_split(all_inputs, all_classes, train_size=0.7, random_state=1)

In [ ]:
dtc = LinearSVC(C=1.0, class_weight='balanced', dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
#dtc = linear_model.SGDClassifier()
#dtc = GaussianNB()
#dtc = BaggingClassifier()
clf=dtc.fit(all_inputs, all_classes)
calibrated_svc = CalibratedClassifierCV(base_estimator=dtc,cv="prefit")
calibrated_svc.fit(all_inputs, all_classes)

In [ ]:
test_inputs = ls[b][100000+num_rep:].values

In [ ]:
ans = calibrated_svc.predict(test_inputs)
ans2 = clf.predict(test_inputs)
print(len(ans))
print(sum(ans))
print()
print(sum(ans2))


In [ ]:
aa = calibrated_svc.predict_proba(test_inputs)
aa

In [ ]:
aaa = pd.DataFrame(aa)
ansss = aaa[1]

In [ ]:
annnnnn = pd.DataFrame({"CUS_ID" : name,"Ypred":ansss })

In [ ]:
annnnnn.to_csv("upload_10_5_3.csv")

In [ ]:
#1 資料重複8次 預測成功數3888
#2 資料重複7次 預測成功數2962

#1 資料重複9次 預測成功數3196
#2 資料重複11次 預測成功數